## Overview
VegaFusion supports pre-transforming datasets within a Vega Spec and then generating a new spec with the transformed data included inline.

**Note**: This is fairly low-level functionality that is not yet integrated with Altair. It is designed to be a building block for larger Vega systems.

In [ ]:
# imports
import json
from altair.vega import vega
import vegafusion as vf
import pandas as pd
import tzlocal

local_tz = tzlocal.get_localzone_name()

## Example 1: JSON url dataset
Consider a simple Vega-Lite histogram like the example at https://vega.github.io/vega-lite/examples/histogram.html

```json
{
  "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
  "data": {"url": "https://raw.githubusercontent.com/vega/vega-datasets/master/data/movies.json"},
  "mark": "bar",
  "encoding": {
    "x": {
      "bin": true,
      "field": "IMDB Rating"
    },
    "y": {"aggregate": "count"}
  }
}

```

This compiles down into the following Vega specification:

In [ ]:
full_spec = r"""
{
  "$schema": "https://vega.github.io/schema/vega/v5.json",
  "background": "white",
  "padding": 5,
  "width": 200,
  "height": 200,
  "style": "cell",
  "data": [
    {
      "name": "source_0",
      "url": "https://raw.githubusercontent.com/vega/vega-datasets/master/data/movies.json",
      "format": {"type": "json"},
      "transform": [
        {
          "type": "extent",
          "field": "IMDB Rating",
          "signal": "bin_maxbins_10_IMDB_Rating_extent"
        },
        {
          "type": "bin",
          "field": "IMDB Rating",
          "as": [
            "bin_maxbins_10_IMDB Rating",
            "bin_maxbins_10_IMDB Rating_end"
          ],
          "signal": "bin_maxbins_10_IMDB_Rating_bins",
          "extent": {"signal": "bin_maxbins_10_IMDB_Rating_extent"},
          "maxbins": 10
        },
        {
          "type": "aggregate",
          "groupby": [
            "bin_maxbins_10_IMDB Rating",
            "bin_maxbins_10_IMDB Rating_end"
          ],
          "ops": ["count"],
          "fields": [null],
          "as": ["__count"]
        },
        {
          "type": "filter",
          "expr": "isValid(datum[\"bin_maxbins_10_IMDB Rating\"]) && isFinite(+datum[\"bin_maxbins_10_IMDB Rating\"])"
        }
      ]
    }
  ],
  "marks": [
    {
      "name": "marks",
      "type": "rect",
      "style": ["bar"],
      "from": {"data": "source_0"},
      "encode": {
        "update": {
          "fill": {"value": "#4c78a8"},
          "ariaRoleDescription": {"value": "bar"},
          "description": {
            "signal": "\"IMDB Rating (binned): \" + (!isValid(datum[\"bin_maxbins_10_IMDB Rating\"]) || !isFinite(+datum[\"bin_maxbins_10_IMDB Rating\"]) ? \"null\" : format(datum[\"bin_maxbins_10_IMDB Rating\"], \"\") + \" – \" + format(datum[\"bin_maxbins_10_IMDB Rating_end\"], \"\")) + \"; Count of Records: \" + (format(datum[\"__count\"], \"\"))"
          },
          "x2": {
            "scale": "x",
            "field": "bin_maxbins_10_IMDB Rating",
            "offset": 1
          },
          "x": {"scale": "x", "field": "bin_maxbins_10_IMDB Rating_end"},
          "y": {"scale": "y", "field": "__count"},
          "y2": {"scale": "y", "value": 0}
        }
      }
    }
  ],
  "scales": [
    {
      "name": "x",
      "type": "linear",
      "domain": {
        "signal": "[bin_maxbins_10_IMDB_Rating_bins.start, bin_maxbins_10_IMDB_Rating_bins.stop]"
      },
      "range": [0, {"signal": "width"}],
      "bins": {"signal": "bin_maxbins_10_IMDB_Rating_bins"},
      "zero": false
    },
    {
      "name": "y",
      "type": "linear",
      "domain": {"data": "source_0", "field": "__count"},
      "range": [{"signal": "height"}, 0],
      "nice": true,
      "zero": true
    }
  ],
  "axes": [
    {
      "scale": "y",
      "orient": "left",
      "gridScale": "x",
      "grid": true,
      "tickCount": {"signal": "ceil(height/40)"},
      "domain": false,
      "labels": false,
      "maxExtent": 0,
      "minExtent": 0,
      "ticks": false,
      "zindex": 0
    },
    {
      "scale": "x",
      "orient": "bottom",
      "grid": false,
      "title": "IMDB Rating (binned)",
      "labelFlush": true,
      "labelOverlap": true,
      "tickCount": {"signal": "ceil(width/10)"},
      "zindex": 0
    },
    {
      "scale": "y",
      "orient": "left",
      "grid": false,
      "title": "Count of Records",
      "labelOverlap": true,
      "tickCount": {"signal": "ceil(height/40)"},
      "zindex": 0
    }
  ]
}
"""

In [ ]:
vega(json.loads(full_spec))

VegaFusion provides a `pre_transform_spec` method that can be used to load the dataset and perform a variety of data transformations in the Python kernel process.

In [ ]:
vf.runtime.pre_transform_spec?

---

In this case, `pre_transform_spec` will load the specified `movies.json` file and then perform the `extent`, `bin`, and `aggregation` transforms.  The dataset that results from applying these transformations is included inline in the returned Vega specification

In [ ]:
transformed_spec, warnings = vf.runtime.pre_transform_spec(full_spec, local_tz)

In [ ]:
print(warnings)

In [ ]:
print(json.dumps(json.loads(transformed_spec), indent=2))

---

When passed to the client for rendering by Vega, this transformed specification will produce the same chart as the original specification. In this case, however, the Vega JavaScript library does not need to load the dataset over the network or perform the `extent`, `bin`, and `aggregate` transforms.

In [ ]:
vega(json.loads(transformed_spec))

## Example 2: DataFrame dataset
Suppose the `movies.json` dataset is already loaded into memory as a pandas `DataFrame`

In [ ]:
movies = pd.read_json("https://raw.githubusercontent.com/vega/vega-datasets/master/data/movies.json")
movies.head()

---

This DataFrame can be provided to `pre_transform_spec` method as a "named dataset", with name `"movies"`.  When a `DataFrame` is provided as a named dataset, it can be referenced by name from within the Vega specification using a special url syntax. In this case, the moveis dataset can be referenced as `"vegafusion+dataset://movies"`.  Here is the full specification:

In [ ]:
full_spec_named = r"""
{
  "$schema": "https://vega.github.io/schema/vega/v5.json",
  "background": "white",
  "padding": 5,
  "width": 200,
  "height": 200,
  "style": "cell",
  "data": [
    {
      "name": "source_0",
      "url": "vegafusion+dataset://movies",
      "transform": [
        {
          "type": "extent",
          "field": "IMDB Rating",
          "signal": "bin_maxbins_10_IMDB_Rating_extent"
        },
        {
          "type": "bin",
          "field": "IMDB Rating",
          "as": [
            "bin_maxbins_10_IMDB Rating",
            "bin_maxbins_10_IMDB Rating_end"
          ],
          "signal": "bin_maxbins_10_IMDB_Rating_bins",
          "extent": {"signal": "bin_maxbins_10_IMDB_Rating_extent"},
          "maxbins": 10
        },
        {
          "type": "aggregate",
          "groupby": [
            "bin_maxbins_10_IMDB Rating",
            "bin_maxbins_10_IMDB Rating_end"
          ],
          "ops": ["count"],
          "fields": [null],
          "as": ["__count"]
        },
        {
          "type": "filter",
          "expr": "isValid(datum[\"bin_maxbins_10_IMDB Rating\"]) && isFinite(+datum[\"bin_maxbins_10_IMDB Rating\"])"
        }
      ]
    }
  ],
  "marks": [
    {
      "name": "marks",
      "type": "rect",
      "style": ["bar"],
      "from": {"data": "source_0"},
      "encode": {
        "update": {
          "fill": {"value": "#4c78a8"},
          "ariaRoleDescription": {"value": "bar"},
          "description": {
            "signal": "\"IMDB Rating (binned): \" + (!isValid(datum[\"bin_maxbins_10_IMDB Rating\"]) || !isFinite(+datum[\"bin_maxbins_10_IMDB Rating\"]) ? \"null\" : format(datum[\"bin_maxbins_10_IMDB Rating\"], \"\") + \" – \" + format(datum[\"bin_maxbins_10_IMDB Rating_end\"], \"\")) + \"; Count of Records: \" + (format(datum[\"__count\"], \"\"))"
          },
          "x2": {
            "scale": "x",
            "field": "bin_maxbins_10_IMDB Rating",
            "offset": 1
          },
          "x": {"scale": "x", "field": "bin_maxbins_10_IMDB Rating_end"},
          "y": {"scale": "y", "field": "__count"},
          "y2": {"scale": "y", "value": 0}
        }
      }
    }
  ],
  "scales": [
    {
      "name": "x",
      "type": "linear",
      "domain": {
        "signal": "[bin_maxbins_10_IMDB_Rating_bins.start, bin_maxbins_10_IMDB_Rating_bins.stop]"
      },
      "range": [0, {"signal": "width"}],
      "bins": {"signal": "bin_maxbins_10_IMDB_Rating_bins"},
      "zero": false
    },
    {
      "name": "y",
      "type": "linear",
      "domain": {"data": "source_0", "field": "__count"},
      "range": [{"signal": "height"}, 0],
      "nice": true,
      "zero": true
    }
  ],
  "axes": [
    {
      "scale": "y",
      "orient": "left",
      "gridScale": "x",
      "grid": true,
      "tickCount": {"signal": "ceil(height/40)"},
      "domain": false,
      "labels": false,
      "maxExtent": 0,
      "minExtent": 0,
      "ticks": false,
      "zindex": 0
    },
    {
      "scale": "x",
      "orient": "bottom",
      "grid": false,
      "title": "IMDB Rating (binned)",
      "labelFlush": true,
      "labelOverlap": true,
      "tickCount": {"signal": "ceil(width/10)"},
      "zindex": 0
    },
    {
      "scale": "y",
      "orient": "left",
      "grid": false,
      "title": "Count of Records",
      "labelOverlap": true,
      "tickCount": {"signal": "ceil(height/40)"},
      "zindex": 0
    }
  ]
}
"""

In [ ]:
transformed_spec, warnings = vf.runtime.pre_transform_spec(
    full_spec_named, local_tz, inline_datasets=dict(movies=movies)
)

In [ ]:
print(warnings)

In [ ]:
print(json.dumps(json.loads(transformed_spec), indent=2))

---
Again, the resulting chart identical when rendered by Vega

In [ ]:
vega(json.loads(transformed_spec))

## Example 3: RowLimit
And optional row limit can be provided to `pre_transform_spec` to set an upper bound on the number of rows that may be included inline in the resulting specification.  If this limit is exceeded, then the dataset will be truncated and a warning object will be returned.

Here is an artificial example of setting the row limit to 4 to demonstrate what happens.

In [ ]:
transformed_spec, warnings = vf.runtime.pre_transform_spec(
    full_spec_named,
    local_tz,
    row_limit=4,
    inline_datasets=dict(movies=movies)
)

In [ ]:
print(warnings)

In [ ]:
vega(json.loads(transformed_spec))

## Example 4: Broken Interactivity
The initial render of the transformed specification should always match the appearance of the initial render to the original specification.  However, it is possible that certain interactive features (e.g. cross filtering) will be broken in the transformed specification. If this happens, a `BrokenInteractivity` warning dict will be included in the warnings list.

The "Interactive Average" Vega-Lite example (https://vega.github.io/vega-lite/examples/selection_layer_bar_month.html) is an example of a specification that will result in broken interactivity.

This example compiles to the following Vega specification:

In [ ]:
full_spec = r"""
{
  "$schema": "https://vega.github.io/schema/vega/v5.json",
  "background": "white",
  "padding": 5,
  "height": 200,
  "style": "cell",
  "data": [
    {"name": "brush_store"},
    {
      "name": "source_0",
      "url": "https://raw.githubusercontent.com/vega/vega-datasets/master/data/seattle-weather.csv",
      "format": {"type": "csv", "parse": {"date": "date"}},
      "transform": [
        {
          "field": "date",
          "type": "timeunit",
          "units": ["month"],
          "as": ["month_date", "month_date_end"]
        }
      ]
    },
    {
      "name": "data_0",
      "source": "source_0",
      "transform": [
        {
          "type": "filter",
          "expr": "!length(data(\"brush_store\")) || vlSelectionTest(\"brush_store\", datum)"
        },
        {
          "type": "aggregate",
          "groupby": [],
          "ops": ["mean"],
          "fields": ["precipitation"],
          "as": ["mean_precipitation"]
        },
        {
          "type": "filter",
          "expr": "isValid(datum[\"mean_precipitation\"]) && isFinite(+datum[\"mean_precipitation\"])"
        }
      ]
    },
    {
      "name": "data_1",
      "source": "source_0",
      "transform": [
        {
          "type": "aggregate",
          "groupby": ["month_date"],
          "ops": ["mean"],
          "fields": ["precipitation"],
          "as": ["mean_precipitation"]
        },
        {
          "type": "filter",
          "expr": "isValid(datum[\"mean_precipitation\"]) && isFinite(+datum[\"mean_precipitation\"])"
        }
      ]
    }
  ],
  "signals": [
    {"name": "x_step", "value": 20},
    {
      "name": "width",
      "update": "bandspace(domain('x').length, 0.1, 0.05) * x_step"
    },
    {
      "name": "unit",
      "value": {},
      "on": [
        {"events": "mousemove", "update": "isTuple(group()) ? group() : unit"}
      ]
    },
    {
      "name": "brush",
      "update": "vlSelectionResolve(\"brush_store\", \"union\")"
    },
    {
      "name": "brush_x",
      "value": [],
      "on": [
        {
          "events": {
            "source": "scope",
            "type": "mousedown",
            "filter": [
              "!event.item || event.item.mark.name !== \"brush_brush\""
            ]
          },
          "update": "[x(unit), x(unit)]"
        },
        {
          "events": {
            "source": "window",
            "type": "mousemove",
            "consume": true,
            "between": [
              {
                "source": "scope",
                "type": "mousedown",
                "filter": [
                  "!event.item || event.item.mark.name !== \"brush_brush\""
                ]
              },
              {"source": "window", "type": "mouseup"}
            ]
          },
          "update": "[brush_x[0], clamp(x(unit), 0, width)]"
        },
        {"events": {"signal": "brush_scale_trigger"}, "update": "[0, 0]"},
        {
          "events": [{"source": "view", "type": "dblclick"}],
          "update": "[0, 0]"
        },
        {
          "events": {"signal": "brush_translate_delta"},
          "update": "clampRange(panLinear(brush_translate_anchor.extent_x, brush_translate_delta.x / span(brush_translate_anchor.extent_x)), 0, width)"
        },
        {
          "events": {"signal": "brush_zoom_delta"},
          "update": "clampRange(zoomLinear(brush_x, brush_zoom_anchor.x, brush_zoom_delta), 0, width)"
        }
      ]
    },
    {
      "name": "brush_month_date",
      "on": [
        {
          "events": {"signal": "brush_x"},
          "update": "brush_x[0] === brush_x[1] ? null : invert(\"x\", brush_x)"
        }
      ]
    },
    {
      "name": "brush_scale_trigger",
      "value": {},
      "on": [
        {
          "events": [{"scale": "x"}],
          "update": "(!isArray(brush_month_date) || (invert(\"x\", brush_x)[0] === brush_month_date[0] && invert(\"x\", brush_x)[1] === brush_month_date[1])) ? brush_scale_trigger : {}"
        }
      ]
    },
    {
      "name": "brush_tuple",
      "on": [
        {
          "events": [{"signal": "brush_month_date"}],
          "update": "brush_month_date ? {unit: \"layer_0\", fields: brush_tuple_fields, values: [brush_month_date]} : null"
        }
      ]
    },
    {
      "name": "brush_tuple_fields",
      "value": [{"field": "month_date", "channel": "x", "type": "E"}]
    },
    {
      "name": "brush_translate_anchor",
      "value": {},
      "on": [
        {
          "events": [
            {"source": "scope", "type": "mousedown", "markname": "brush_brush"}
          ],
          "update": "{x: x(unit), y: y(unit), extent_x: slice(brush_x)}"
        }
      ]
    },
    {
      "name": "brush_translate_delta",
      "value": {},
      "on": [
        {
          "events": [
            {
              "source": "window",
              "type": "mousemove",
              "consume": true,
              "between": [
                {
                  "source": "scope",
                  "type": "mousedown",
                  "markname": "brush_brush"
                },
                {"source": "window", "type": "mouseup"}
              ]
            }
          ],
          "update": "{x: brush_translate_anchor.x - x(unit), y: brush_translate_anchor.y - y(unit)}"
        }
      ]
    },
    {
      "name": "brush_zoom_anchor",
      "on": [
        {
          "events": [
            {
              "source": "scope",
              "type": "wheel",
              "consume": true,
              "markname": "brush_brush"
            }
          ],
          "update": "{x: x(unit), y: y(unit)}"
        }
      ]
    },
    {
      "name": "brush_zoom_delta",
      "on": [
        {
          "events": [
            {
              "source": "scope",
              "type": "wheel",
              "consume": true,
              "markname": "brush_brush"
            }
          ],
          "force": true,
          "update": "pow(1.001, event.deltaY * pow(16, event.deltaMode))"
        }
      ]
    },
    {
      "name": "brush_modify",
      "on": [
        {
          "events": {"signal": "brush_tuple"},
          "update": "modify(\"brush_store\", brush_tuple, true)"
        }
      ]
    }
  ],
  "marks": [
    {
      "name": "brush_brush_bg",
      "type": "rect",
      "clip": true,
      "encode": {
        "enter": {"fill": {"value": "#333"}, "fillOpacity": {"value": 0.125}},
        "update": {
          "x": [
            {
              "test": "data(\"brush_store\").length && data(\"brush_store\")[0].unit === \"layer_0\"",
              "signal": "brush_x[0]"
            },
            {"value": 0}
          ],
          "y": [
            {
              "test": "data(\"brush_store\").length && data(\"brush_store\")[0].unit === \"layer_0\"",
              "value": 0
            },
            {"value": 0}
          ],
          "x2": [
            {
              "test": "data(\"brush_store\").length && data(\"brush_store\")[0].unit === \"layer_0\"",
              "signal": "brush_x[1]"
            },
            {"value": 0}
          ],
          "y2": [
            {
              "test": "data(\"brush_store\").length && data(\"brush_store\")[0].unit === \"layer_0\"",
              "field": {"group": "height"}
            },
            {"value": 0}
          ]
        }
      }
    },
    {
      "name": "layer_0_marks",
      "type": "rect",
      "style": ["bar"],
      "interactive": true,
      "from": {"data": "data_1"},
      "encode": {
        "update": {
          "fill": {"value": "#4c78a8"},
          "opacity": [
            {
              "test": "!length(data(\"brush_store\")) || vlSelectionTest(\"brush_store\", datum)",
              "value": 1
            },
            {"value": 0.7}
          ],
          "ariaRoleDescription": {"value": "bar"},
          "description": {
            "signal": "\"date (month): \" + (timeFormat(datum[\"month_date\"], timeUnitSpecifier([\"month\"], {\"year-month\":\"%b %Y \",\"year-month-date\":\"%b %d, %Y \"}))) + \"; Mean of precipitation: \" + (format(datum[\"mean_precipitation\"], \"\"))"
          },
          "x": {"scale": "x", "field": "month_date"},
          "width": {"scale": "x", "band": 1},
          "y": {"scale": "y", "field": "mean_precipitation"},
          "y2": {"scale": "y", "value": 0}
        }
      }
    },
    {
      "name": "layer_1_marks",
      "type": "rule",
      "style": ["rule"],
      "interactive": false,
      "from": {"data": "data_0"},
      "encode": {
        "update": {
          "stroke": {"value": "firebrick"},
          "description": {
            "signal": "\"Mean of precipitation: \" + (format(datum[\"mean_precipitation\"], \"\"))"
          },
          "x": {"field": {"group": "width"}},
          "x2": {"value": 0},
          "y": {"scale": "y", "field": "mean_precipitation"},
          "strokeWidth": {"value": 3}
        }
      }
    },
    {
      "name": "brush_brush",
      "type": "rect",
      "clip": true,
      "encode": {
        "enter": {"fill": {"value": "transparent"}},
        "update": {
          "x": [
            {
              "test": "data(\"brush_store\").length && data(\"brush_store\")[0].unit === \"layer_0\"",
              "signal": "brush_x[0]"
            },
            {"value": 0}
          ],
          "y": [
            {
              "test": "data(\"brush_store\").length && data(\"brush_store\")[0].unit === \"layer_0\"",
              "value": 0
            },
            {"value": 0}
          ],
          "x2": [
            {
              "test": "data(\"brush_store\").length && data(\"brush_store\")[0].unit === \"layer_0\"",
              "signal": "brush_x[1]"
            },
            {"value": 0}
          ],
          "y2": [
            {
              "test": "data(\"brush_store\").length && data(\"brush_store\")[0].unit === \"layer_0\"",
              "field": {"group": "height"}
            },
            {"value": 0}
          ],
          "stroke": [
            {"test": "brush_x[0] !== brush_x[1]", "value": "white"},
            {"value": "transparent"}
          ]
        }
      }
    }
  ],
  "scales": [
    {
      "name": "x",
      "type": "band",
      "domain": {"data": "data_1", "field": "month_date", "sort": true},
      "range": {"step": {"signal": "x_step"}},
      "paddingInner": 0.1,
      "paddingOuter": 0.05
    },
    {
      "name": "y",
      "type": "linear",
      "domain": {
        "fields": [
          {"data": "data_1", "field": "mean_precipitation"},
          {"data": "data_0", "field": "mean_precipitation"}
        ]
      },
      "range": [{"signal": "height"}, 0],
      "nice": true,
      "zero": true
    }
  ],
  "axes": [
    {
      "scale": "y",
      "orient": "left",
      "gridScale": "x",
      "grid": true,
      "tickCount": {"signal": "ceil(height/40)"},
      "domain": false,
      "labels": false,
      "maxExtent": 0,
      "minExtent": 0,
      "ticks": false,
      "zindex": 0
    },
    {
      "scale": "x",
      "orient": "bottom",
      "grid": false,
      "title": "date (month)",
      "format": {
        "signal": "timeUnitSpecifier([\"month\"], {\"year-month\":\"%b %Y \",\"year-month-date\":\"%b %d, %Y \"})"
      },
      "formatType": "time",
      "labelOverlap": true,
      "zindex": 0
    },
    {
      "scale": "y",
      "orient": "left",
      "grid": false,
      "title": "Mean of precipitation",
      "labelOverlap": true,
      "tickCount": {"signal": "ceil(height/40)"},
      "zindex": 0
    }
  ]
}
"""

In [ ]:
vega(json.loads(full_spec))

In [ ]:
transformed_spec, warnings = vf.runtime.pre_transform_spec(
    full_spec,
    local_tz,
    inline_datasets=dict(movies=movies)
)

In [ ]:
print(warnings)

In [ ]:
vega(json.loads(transformed_spec))

In [ ]:
print(json.dumps(json.loads(transformed_spec), indent=2))